In [73]:
## 0.A Import libraries
import os

# dates
from datetime import datetime

# data handling
import numpy as np
import pandas as pd

# Data viz
%matplotlib inline
import matplotlib.pyplot as plt

# custom
from utils.custom_functions import prepare_gdp_file

ImportError: cannot import name 'prepare_gdp_file' from 'utils.custom_functions' (/Users/clement/Desktop/ULB/Doctorat/ECARES M2/Advanced topics in Economics/Project/gravity_trade/src/utils/custom_functions.py)

In [6]:
## O.C Arguments Paths
gravity_filename = r"gravity_1995_2014.dta"

trade_filename = r"TradeData.csv"

gdp_filename_1 = r"gdp_1.csv"
gdp_filename_2 = r"gdp_2.csv"

population_filename = r"pop.csv"

country_code_filename = r"iso-country-codes.csv"

tweets_refactored_filename = r"grouped_refactored.csv"

In [1]:
folder_path = os.path.dirname(os.getcwd())
path_to_data = os.path.join(folder_path,'data')

## Load ALL

In [129]:
country_code = pd.read_csv(os.path.join(path_to_data, country_code_filename))
country_code.rename(columns = {'English short name lower case' : 'Country'}, inplace = True)
country_code = country_code[['Country', 'Alpha-3 code']]

In [130]:
# Load Trade data
trade = pd.read_csv(os.path.join(path_to_data, trade_filename))
trade.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf4 in position 24329: invalid continuation byte

In [131]:
mapping_years = {'2012 [YR2012]': '2012',
'2013 [YR2013]': '2013',
'2014 [YR2014]': '2014',
'2015 [YR2015]': '2015',
'2016 [YR2016]': '2016',
'2017 [YR2017]': '2017',
'2018 [YR2018]': '2018',
'2019 [YR2019]': '2019',
'2020 [YR2020]': '2020',
'2021 [YR2021]': '2021',
'2022 [YR2022]': '2022'
}

In [132]:
# Load gdp data
def prepare_gdp_file(path_to_data, gdp_filename): 
    """Refactofing file.

    Args:
        path_to_data (_type_): _description_
        gdp_filename (_type_): _description_
    """
    gdp1 = pd.read_csv(os.path.join(path_to_data, gdp_filename))
    gdp1 = gdp1[gdp1["Series Name"] == "GDP (current US$)"]
    gdp1.rename(columns=mapping_years, inplace=True)
    gdp1 = gdp1.drop(columns=['Country Name','Series Name', 'Series Code'])
    gdp1 = gdp1.set_index('Country Code')
    gdp1 = gdp1.stack().reset_index()
    gdp1.rename(columns = {'level_1':'year', 0:'gdp'}, inplace=True)
    gdp1['Key'] = gdp1['Country Code'] + '_' + gdp1['year'].astype(str)
    
    return gdp1

In [133]:
gdp1 = prepare_gdp_file(path_to_data, gdp_filename_1)

In [134]:
# load pop data
pop = pd.read_csv(os.path.join(path_to_data, population_filename))
pop = pop[pop["Series Name"] == "Population, total"]
pop.rename(columns=mapping_years, inplace=True)
pop = pop.drop(columns=['Country Name','Series Name', 'Series Code'])
pop = pop.set_index('Country Code')
pop = pop.stack().reset_index()
pop.rename(columns = {'level_1':'year', 0:'population'}, inplace=True)
pop['Key'] = pop['Country Code'] + '_' + pop['year'].astype(str)

In [135]:
# Load Gravity data
gravity = pd.read_stata(os.path.join(path_to_data, gravity_filename))
gravity.t = gravity.t.astype(int)
gravity = gravity[gravity['i']=='CHN']
distance = gravity[['j','dist']].drop_duplicates()
distance.head()

,j,dist
82529,ABW,14155.350
82548,AFG,4180.438
82568,AGO,11769.510
82588,AIA,13563.450
82596,ALB,7686.079


In [136]:
gravity = gravity[gravity['i']=='CHN']
distance = gravity[['j','dist']].drop_duplicates()

In [144]:
# join gdp and pop and dist
data = pd.merge(gdp1, pop[['population', 'Key']], on = 'Key', how = 'outer')
data.head()

,Country Code,year,gdp,Key,population
0,AFG,2012,20203572959.5023,AFG_2012,30466479
1,AFG,2013,20564485419.1684,AFG_2013,31541209
2,AFG,2014,20550582746.8448,AFG_2014,32716210
3,AFG,2015,19998156214.3988,AFG_2015,33753499
4,AFG,2016,18019558181.9602,AFG_2016,34636207


In [145]:
data = pd.merge(data, distance, left_on = 'Country Code', right_on='j', how = 'left')
data.dropna(subset = 'Country Code', inplace = True)
data

,Country Code,year,gdp,Key,population,j,dist
0,AFG,2012,20203572959.5023,AFG_2012,30466479,AFG,4180.438
1,AFG,2013,20564485419.1684,AFG_2013,31541209,AFG,4180.438
2,AFG,2014,20550582746.8448,AFG_2014,32716210,AFG,4180.438
3,AFG,2015,19998156214.3988,AFG_2015,33753499,AFG,4180.438
4,AFG,2016,18019558181.9602,AFG_2016,34636207,AFG,4180.438
...,...,...,...,...,...,...,...
2655,ZWE,2017,17584890937.482,ZWE_2017,14751101,ZWE,10898.120
2656,ZWE,2018,34156069918.0609,ZWE_2018,15052184,ZWE,10898.120
2657,ZWE,2019,21832234925.5021,ZWE_2019,15354608,ZWE,10898.120
2658,ZWE,2020,21509698406.1116,ZWE_2020,15669666,ZWE,10898.120


In [149]:
# Add Twitter data
tweets = pd.read_csv(os.path.join(path_to_data,tweets_refactored_filename), sep = ';')
tweets['Key'] = tweets['Alpha-3 code'] + '_' + tweets['year'].astype(str)
tweets = tweets[['sum_pos_tweets','count_tweets','sum_political_tweets','sum_likes','sum_retweeets', 'Key']]
tweets

,sum_pos_tweets,count_tweets,sum_political_tweets,sum_likes,sum_retweeets,Key
0,2,4,4,0,6,AFG_2013
1,8,18,18,10,15,AFG_2014
2,6,11,11,1,0,AFG_2015
3,142,468,468,459,194,AFG_2016
4,1923,4956,4956,6618,6953,AFG_2017
...,...,...,...,...,...,...
1066,1143,3001,3001,2491,2008,ZWE_2017
1067,4010,9240,9240,19549,8933,ZWE_2018
1068,13077,28903,28903,62871,30636,ZWE_2019
1069,7665,17976,17976,55590,18021,ZWE_2020


In [151]:
data = pd.merge(data, tweets, left_on = 'Key', right_on='Key', how = 'left')
data.to_csv('data_witout_trade.csv')

In [ ]:
# load trade
